In [ ]:
# import packages

import numpy as np
np.random.seed(87)

import keras
from keras.datasets import cifar10
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D



In [ ]:
# 新聞標題 (文字應用的NN)
from keras.layers import Input, Dense
from keras.layers import Embedding  # Embedding 層將輸入序列編碼為一個稠密向量的序列
from keras.layers import LSTM       # LSTM 層把向量序列轉換成單個向量，它包含整個序列的上下文信息
from keras.models import Model



In [ ]:
# 設定一些參數
batch_size = 32
num_classes = 10
epochs = 10



# 資料準備 / 取得資料，並分成 Training 與 Test set
(x_img_train,y_label_train), (x_img_test, y_label_test) = cifar10.load_data()

# 確認資料維度
print("train data:",'images:', x_img_train.shape, " labels:", y_label_train.shape) 
print("test  data:",'images:', x_img_test.shape , " labels:", y_label_test.shape) 


# 資料(影像)正規化 (Image normalization)，並設定 data array 為浮點數
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

# 轉換 label 為 OneHot Encoding (只要是做分類問題的時候，都要對 label 做 one-hot encoding喔！) Convert class vectors to binary class matrices.
# 針對 Label 做 ONE HOT Encoding, 並查看維度資訊
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
print(y_label_test_OneHot.shape)

## MLP (多層 Perceptron) 的優缺點

MLP 優點

> 建立 「非線性」 模型、 「real-time」 模型

MLP 缺點

>使用不同的初始權重，會讓驗證時的準確率浮動 MLP 需要調整每層神經元數、層數、迭代次數 對於特徵預處理很敏感

In [ ]:
# 建立模型
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
# build model
"""
NN的組成

1. Input Layer

    Weights

2. Hidden Layer

    Net Input Function

    Activation Function : 給神經元引入非線性因素，使得 神經網路能任意逼近任何非線性函數

3. Output Layer
"""







"""# model - 1"""
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))



"""# model - 2"""
# 宣告採用序列模型
model = Sequential()

# 卷積層1 - filters=32
# 與池化層1
model.add(Conv2D(filters=32,
                 kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 卷積層2 - filters=64
# 與池化層2
model.add(Conv2D(filters=64, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 卷積層3 - filters=128
# 與池化層3
model.add(Conv2D(filters=128, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 卷積層4 - filters=256
# 與池化層4
model.add(Conv2D(filters=256, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

"""
# 建立神經網路(平坦層、隱藏層、輸出層)
"""
""" 1. Flatten layer (Input Layer) """
model.add(Flatten())

""" 2. Fully Connected Layer (Hidden Layer) """
# 建立全網路連接層
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))

""" 3. Output Layer """
#建立輸出層
model.add(Dense(10, activation='softmax'))

#檢查model 的STACK
print(model.summary())












In [ ]:
# 載入之前訓練的模型

try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")







In [ ]:
# compile model (模型編譯)
"""
# 模型編譯

在訓練模型之前，您需要配置學習過程，的英文這通過compile方法完成的它接收三個參數：

1. Optimizer

優化器optimizer。它可以是現有優化器的字符串標識符，如rmsprop或adagrad，也可以是Optimizer類的實例。

2. Loss

損失函數的損失，模型試圖最小化的目標函數它可以是現有損失函數的字符串標識符，如。categorical_crossentropy或mse，也可以是一個目標函數

3. metrics 評估指標

評估標準指標。對於任何分類問題，你都希望將其設置為metrics = ['accuracy']。評估標準可以是現有的標準的字符串標識符，也可以是自定義的評估標準函數。
"""


model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])






In [ ]:
# fit(train) model


#模型訓練, "Train_History" 把訓練過程所得到的數值存起來
train_history = model.fit(x_img_train_normalize, y_label_train_OneHot,
                          validation_split=0.25,
                          epochs=12, 
                          batch_size=128, 
                          verbose=1
                         )         

# [validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的比例。
# 該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
# [batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128

In [ ]:
# 畫出 train acc, validation acc，以檢視是否有 overfitting / underfitting

import matplotlib.pyplot as plt
%matplotlib inline

# 定義一個繪圖函數
def show_train_history(train_acc, test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Train History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    plt.show()
    
show_train_history('acc', 'val_acc')

In [ ]:
# scores = model.evaluate(x_Test_normalize, y_Test_OneHot)

scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print()
print('accuracy=',scores[1])

In [ ]:
# 結合 compile, fit, plot, scores

def check_model(loss_function) :
        
    model.compile(loss = loss_function, optimizer='sgd', metrics=['accuracy'])

    #模型訓練, "Train_History" 把訓練過程所得到的數值存起來
    train_history = model.fit(x_img_train_normalize, 
                              y_label_train_OneHot,
                              validation_split=0.25,
                              epochs=12, 
                              batch_size=128, 
                              verbose=1
                             )         

    # [validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的比例。
    # 該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
    # [batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128

    # 定義一個繪圖函數
    def show_train_history(train_history, train, validation):
        plt.plot(train_history.history[train])
        plt.plot(train_history.history[validation])
        plt.title('Train History')
        plt.ylabel(train)
        plt.xlabel('Epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()
        
    # plot
    show_train_history(train_history, 'acc', 'val_acc')
    show_train_history(train_history, 'loss', 'val_loss')

    # scores
    scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
    print()
    print('accuracy=',scores[1])
    
    
    
#check_model(loss_function)
check_model('categorical_crossentropy')
check_model('mean_squared_error')